In [ ]:
install.packages("parsnip")
install.packages('dotwhisker')
install.packages('nycflights13')
install.packages('rsample')
install.packages('recipes')
install.packages('workflows')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘globals’, ‘hardhat’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘bayestestR’, ‘datawizard’, ‘insight’, ‘prediction’, ‘plyr’, ‘parameters’, ‘margins’, ‘ggstance’


Warning message in download.file(url, destfile, method, mode = "wb", ...):
“cannot open URL 'https://cran.rstudio.com/src/contrib/parameters_0.20.2.tar.gz': HTTP status was '403 Forbidden'”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.rstudio.com/src/contrib/parameters_0.20.2.tar.gz'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘parameters’ failed”
Warning message in install.packages("dotwhisker"):
“installation of package ‘dotwhisker’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘warp’, ‘furrr’, ‘slider’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘modelenv’




In [ ]:
library(workflows)
library(recipes)
library(rsample)
library(nycflights13)
library(parsnip)
library(modelr)
library(tidyverse)  
library(readr)       # for importing data
library(dotwhisker)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘recipes’


The following object is masked from ‘package:stats’:

    step


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ stringr 1.4.1
✔ tidyr   1.3.0     ✔ forcats 1.0.0
✔ readr   2.1.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ stringr::fixed() masks recipes::fixed()
✖ dplyr::lag()     masks stats::lag()



ERROR: ignored

# Lecture 21: Data processing with recipes

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will understand how to build a recipe with steps such as:**
* Creating dummy variables
* Transforming values to different scales
* Add new features through feature engineering etc..


</div>

These notes follow Chapter 2 of [Tidy Models](https://www.tidymodels.org/start/)

### Predict if a flight is going to be late or not




In [ ]:
set.seed(123)

flight_data <- 
  flights %>% 
  mutate(
    # Convert the arrival delay to a factor
    arr_delay = factor(ifelse(arr_delay >= 30, "late", "on_time")),
    # We will use the date (not date-time) in the recipe below
    date = lubridate::as_date(time_hour)
  ) %>% 
  # Include the weather data
  inner_join(weather, by = c("origin", "time_hour")) %>% 
  # Only retain the specific columns we will use
  select(dep_time, flight, origin, dest, air_time, distance, 
         carrier, date, arr_delay, time_hour) %>% 
  # Exclude missing data
  na.omit() %>% 
  # For creating models, it is better to have qualitative columns
  # encoded as factors (instead of character strings)
  mutate_if(is.character, as.factor)

flight_data %>% print

# A tibble: 325,819 × 10
   dep_time flight origin dest  air_time distance carrier date       arr_delay
      <int>  <int> <fct>  <fct>    <dbl>    <dbl> <fct>   <date>     <fct>    
 1      517   1545 EWR    IAH        227     1400 UA      2013-01-01 on_time  
 2      533   1714 LGA    IAH        227     1416 UA      2013-01-01 on_time  
 3      542   1141 JFK    MIA        160     1089 AA      2013-01-01 late     
 4      544    725 JFK    BQN        183     1576 B6      2013-01-01 on_time  
 5      554    461 LGA    ATL        116      762 DL      2013-01-01 on_time  
 6      554   1696 EWR    ORD        150      719 UA      2013-01-01 on_time  
 7      555    507 EWR    FLL        158     1065 B6      2013-01-01 on_time  
 8      557   5708 LGA    IAD         53      229 EV      2013-01-01 on_time  
 9      557     79 JFK    MCO        140      944 B6      2013-01-01 on_time  
10      558    301 LGA    ORD        138      733 AA      2013-01-01 on_time  
# ℹ 325,809 more rows
# ℹ 1

Lets find the proportion of late flights

In [ ]:
flight_data %>% 
  count(arr_delay) %>% 
  mutate(prop = n/sum(n))

arr_delay,n,prop
<fct>,<int>,<dbl>
late,52540,0.1612552
on_time,273279,0.8387448


### Case for Logistic Regression
The outcome column here is arr_delay. This usecase needs us to apply Logistic Regression as we need a factor output for this outcome column


<img src='https://ebooks.mobibootcamp.com/ml_ai/model-building.png' />

#### Train/Test Split

To build a model, we split the given labelled data into train and test datasets. 

We take the:
* Train set to fit the model (typically 70-80% of the data is kept aside)
* Test set to calculate the model accuracy (balance 20-30%)


In [ ]:

set.seed(222)
# Put 3/4 of the data into the training set 
data_split <- initial_split(flight_data, prop = 3/4)

# Create data frames for the two sets:
train_data <- training(data_split)
test_data  <- testing(data_split)

## Create the recipe



In [ ]:
flights_rec <- 
  recipe(arr_delay ~ ., data = train_data)

The recipe() function arguments are:

* A formula. Any variable on the left-hand side of the tilde (~) is considered the model outcome (here, arr_delay). On the right-hand side of the tilde are the predictors. Variables may be listed by name, or you can use the dot (.) to indicate all other variables as predictors.
* The data. The set data is used to catalog the names of the variables and their types, like factors, integers, dates, etc.



### Feature exclusion

There are two variables that we don’t want to use as predictors in our model, but we would like to retain as identification variables that can be used to troubleshoot late. These are 
* flight, a numeric value
* time_hour, a date-time value.

We can use the update_role() function to let recipes know that flight and time_hour are variables with a custom role that we called "ID" (a role can have any character value). 


In [ ]:
flights_rec <- 
  recipe(arr_delay ~ ., data = train_data) %>% 
  update_role(flight, time_hour, new_role = "ID") 

This step of adding roles to a recipe is optional; the purpose of using it here is that those two variables can be retained in the data but not included in the model.

### Feature Engineering

Perhaps it is reasonable to think that the following information corresponding to the flight date may have an effect to the outcome

* the day of the week (dow)
*  the month
* whether or not the date corresponds to a holiday.

However, we do not have this information readymade in our dataset. But we can derive these from the `date` column. Lets add this step in our recipe

In [ ]:
flights_rec <- 
  recipe(arr_delay ~ ., data = train_data) %>% 
  update_role(flight, time_hour, new_role = "ID") %>% 
  step_date(date, features = c("dow", "month")) %>%               
  step_holiday(date, 
               holidays = timeDate::listHolidays("US"), 
               keep_original_cols = FALSE)


* With step_date(), we created two new factor columns; day of the week (dow) and the month.
* With step_holiday(), we created a binary variable indicating whether the current date is a holiday or not. The argument value of timeDate::listHolidays("US") uses the timeDate package to list the 17 standard US holidays.
* With keep_original_cols = FALSE, we remove the original date variable since we no longer want it in the model. Many recipe steps that create new variables have this argument.

### Creating dummy variables

Recall, last week we saw how the distinct nominal values of 'x' column; 'a', 'b', 'c', 'd' from sim2 data were encoded with 1, 0 values across three new columns that were created by the model



In [ ]:
sim2 %>% model_matrix(y ~ x) %>% head()

(Intercept),xb,xc,xd
<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0
1,0,0,0
1,0,0,0
1,0,0,0
1,0,0,0
1,0,0,0


We need to create similar dummy variables for `dest`, `origin` columns and any other nominal column of our flights dataset

Let us now add this step in our recipe


In [ ]:
flights_rec <- 
  recipe(arr_delay ~ ., data = train_data) %>% 
  update_role(flight, time_hour, new_role = "ID") %>% 
  step_date(date, features = c("dow", "month")) %>%               
  step_holiday(date, 
               holidays = timeDate::listHolidays("US"), 
               keep_original_cols = FALSE) %>% 
  step_dummy(all_nominal_predictors()) %>% print



── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 7
ID:        2



── Operations 

• Date features from: date

• Holiday features from: date

• Dummy variables from: all_nominal_predictors()



### Data cleaning

`carrier` and `dest` have some infrequently occurring factor values, it is possible that dummy variables might be created for values that don’t exist in the training set.

In [ ]:
test_data %>% 
  distinct(dest) %>% 
  anti_join(train_data)

Joining with `by = join_by(dest)`


dest
<fct>
LEX


During dummy variables creation step, a column for LEX is added as factor levels are derived from full dataset, but this column will contain all zeros in the training set. 

This is a “zero-variance predictor” that has no information within the column. While some R functions will not produce an error for such predictors, it usually causes warnings and other issues. `step_zv()` will remove columns from the data when the training set data have a single value, so it is added to the recipe after step_dummy()

In [ ]:
flights_rec <- 
  recipe(arr_delay ~ ., data = train_data) %>% 
  update_role(flight, time_hour, new_role = "ID") %>% 
  step_date(date, features = c("dow", "month")) %>%               
  step_holiday(date, 
               holidays = timeDate::listHolidays("US"), 
               keep_original_cols = FALSE) %>% 
  step_dummy(all_nominal_predictors()) %>% 
  step_zv(all_predictors())

**Finally our recipe is ready!**

### Fit a model with the recipe

We will now apply logistic regression to model the flight data so let us first get our model

In [ ]:
lr_mod <- 
  logistic_reg() %>% 
  set_engine("glm")

We can now use a model workflow, which pairs a model and recipe together. 
Using workflow is a better approach because 
* different recipes are often needed for different models
* when a model and recipe are bundled, it becomes easier to train and test workflows.


In [ ]:

flights_wflow <- 
  workflow() %>% 
  add_model(lr_mod) %>% 
  add_recipe(flights_rec)

## Finally fit the model

In [ ]:
flights_fit <- 
  flights_wflow %>% 
  fit(data = train_data)

Now let us find the co-efficients


In [ ]:
flights_fit %>% 
  extract_fit_parsnip() %>% 
  tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),7.264443103,2.728356e+00,2.6625717,7.754605e-03
dep_time,-0.001664485,1.409802e-05,-118.0650977,0.000000e+00
air_time,-0.044033270,5.628809e-04,-78.2283903,0.000000e+00
distance,0.005083028,1.501843e-03,3.3845261,7.130125e-04
date_USChristmasDay,1.346473737,1.775144e-01,7.5851511,3.320999e-14
date_USColumbusDay,0.720543275,1.703018e-01,4.2309791,2.326762e-05
date_USCPulaskisBirthday,0.804450853,1.391300e-01,5.7820092,7.381369e-09
date_USDecorationMemorialDay,0.581697409,1.173892e-01,4.9552870,7.222361e-07
date_USElectionDay,0.944946570,1.901687e-01,4.9689914,6.730205e-07


## So what exactly did we do till now?


* Built the model (lr_mod)

* Created a preprocessing recipe (flights_rec)

* Bundled the model and recipe (flights_wflow)

* Trained our workflow using a single call to fit()

### Finally prediction
Now we are ready to predict! The test data that is kept aside is used for testing our model's accuracy


In [ ]:
predict(flights_fit, test_data) %>% print

# A tibble: 81,455 × 1
   .pred_class
   <fct>      
 1 on_time    
 2 on_time    
 3 on_time    
 4 on_time    
 5 on_time    
 6 on_time    
 7 on_time    
 8 on_time    
 9 on_time    
10 on_time    
# ℹ 81,445 more rows


If we also want the  predicted class probabilities for each flight, we can specify `type = "prob"` when we use predict() or use `augment()` with the model plus test data to save them together:

In [ ]:
flights_aug <- 
  augment(flights_fit, test_data)

# The data look like: 
predicted <- flights_aug %>%
  select(arr_delay, time_hour, flight, .pred_class, .pred_on_time) %>% print

# A tibble: 81,455 × 5
   arr_delay time_hour           flight .pred_class .pred_on_time
   <fct>     <dttm>               <int> <fct>               <dbl>
 1 on_time   2013-01-01 05:00:00   1545 on_time             0.945
 2 on_time   2013-01-01 05:00:00   1714 on_time             0.949
 3 on_time   2013-01-01 06:00:00    507 on_time             0.964
 4 on_time   2013-01-01 06:00:00   5708 on_time             0.961
 5 on_time   2013-01-01 06:00:00     71 on_time             0.962
 6 on_time   2013-01-01 06:00:00    194 on_time             0.975
 7 on_time   2013-01-01 06:00:00   1124 on_time             0.963
 8 on_time   2013-01-01 05:00:00   1806 on_time             0.981
 9 on_time   2013-01-01 06:00:00   1187 on_time             0.935
10 on_time   2013-01-01 06:00:00   4650 on_time             0.931
# ℹ 81,445 more rows


### Accuracy measurement

In [ ]:
flights_aug %>% mutate(correct_pred = arr_delay == .pred_class) %>% 
  count(correct_pred)

correct_pred,n
<lgl>,<int>
FALSE,12300
TRUE,69155
